In [ ]:
!ls

In [ ]:
import os
import keras
from keras.models import model_from_json
from scipy import misc, ndimage
import imageio
import skimage
import numpy as np
from keras.layers import Conv2D, Activation
from keras.models import Model
from keras.models import Sequential
from layers_builder import Interp
import layers_builder as layers

### Load model

In [ ]:
# print("Loanding model...")
# weights = "pspnet101_cityscapes"

# # json_path = os.path.join("weights", "keras", weights + ".json")
# # h5_path = os.path.join("weights", "keras", weights + ".h5")
# h5_path = "/home/thaovt7/workspace/PSPNet-Keras-tensorflow/weights_train/weight_cityscapes_last_layer/weights.150-20.10.h5"

# # with open(json_path, 'r') as file_handle:
# #     model = model_from_json(file_handle.read())
# model = layers.build_pspnet(nb_classes=5,
#                             resnet_layers=101,
#                             input_shape=(713,713))

# model.load_weights(h5_path)
# print(model.summary())

In [ ]:
print("Loanding model...")
weights = "pspnet101_cityscapes"

h5_path = "/home/thaovt7/workspace/PSPNet-Keras-tensorflow/weights_train/weight_cityscapes_last_layer/weights.150-20.10.h5"
model = layers.build_pspnet(nb_classes=5,
                            resnet_layers=101,
                            input_shape=(713,713))
model.load_weights(h5_path)

new_model = layers.build_pspnet(nb_classes=5,
                            resnet_layers=101,
                            input_shape=(233,233))
new_model.set_weights(model.get_weights())
# for layer in new_model.layers[:382]:
#     layer.trainable = False
print(new_model.summary())


### Test prediction

In [ ]:
from skimage.transform import rescale, resize

img_path = "/home/thaovt7/workspace/Dataset/imgs_test/cityscapes/aachen_000020_000019_leftImg8bit.png"
# "/home/thaovt7/workspace/Dataset/imgs/annotation_1/20_0_202id_mah04071_00h02m33s_004591.png.png"
input_size = 233
print("Processing image {}".format(img_path))
img = misc.imread(img_path, mode='RGB')
# img = imageio.imread(img_path, pilmode="RGB")
# cimg = misc.imresize(img, (input_size, input_size))
cimg = misc.imresize(img, (233, 233))
# cimg = resize(img, (input_size, input_size), mode='reflect', anti_aliasing=False)


In [ ]:
img.shape

In [ ]:
# These are the means for the ImageNet pretrained ResNet
DATA_MEAN = np.array([[[123.68, 116.779, 103.939]]])  # RGB order

cimg = cimg - DATA_MEAN
cimg = cimg[:, :, ::-1]  # RGB => BGR
cimg = cimg.astype('float32')

In [ ]:
# cimg1 = cimg[-234:-1,-234:-1,:]
# # cimg1 = cimg[-234:-1,:233,:]
# # cimg1 = cimg[:233,:233,:]
# # cimg1 = cimg[-234:-1,250:250+233,:]

In [ ]:
# cimg1.shape

In [ ]:
print("Predicting...")
prediction = new_model.predict(np.expand_dims(cimg, 0))[0]

In [ ]:
h_ori, w_ori = img.shape[:2]
h, w = prediction.shape[:2]
# prediction = ndimage.zoom(prediction, (1. * h_ori / h, 1. * w_ori / w, 1.),
#                                  order=1, prefilter=False)
prediction = ndimage.zoom(prediction, (1. * h_ori / (h), 1. * w_ori / (w), 1.),
                                 order=1, prefilter=False)
cimg = cimg + DATA_MEAN
cimg = cimg[:, :, ::-1]  # RGB => BGR
cimg = cimg.astype('float32')
recimg = misc.imresize(cimg, (341, 683))

In [ ]:
# h_ori, w_ori = img.shape[:2]
# h, w = prediction.shape[:2]
# # prediction = ndimage.zoom(prediction, (1. * h_ori / h, 1. * w_ori / w, 1.),
# #                                  order=1, prefilter=False)
# prediction = ndimage.zoom(prediction, (1. * h_ori / (h), 1. * w_ori / (w), 1.),
#                                  order=1, prefilter=False)
# cimg1 = cimg1 + DATA_MEAN
# cimg1 = cimg1[:, :, ::-1]  # RGB => BGR
# cimg1 = cimg1.astype('float32')
# recimg = misc.imresize(cimg1, (341, 683))

In [ ]:
prediction.shape

In [ ]:
cm = np.argmax(prediction, axis=2)

In [ ]:
np.unique(cm)

In [ ]:
from python_utils import utils
color_cm = utils.add_color(cm)


In [ ]:
np.unique(color_cm)

In [ ]:
misc.imsave("result_5_713_150epoch.jpg",color_cm)

# Display images

In [ ]:

from PIL import Image
im = Image.fromarray(np.uint8(recimg))
im.show()

### display mask

In [ ]:
from PIL import Image
mask_im = Image.fromarray(np.uint8(color_cm))
mask_im.show()

In [ ]:
im.resize??

In [ ]:
/home/thaovt7/workspace/Dataset/imgs/annotation/20_0_202id_mah04071_00h18m23s_033075.png.png